<a href="https://colab.research.google.com/github/AnIsAsPe/LDATopicModeling_pyspark/blob/main/Modelos_de_lenguaje_usando_n_gramas_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [28]:
from collections import Counter
import random 


## Funciones

In [3]:
def leer_libro(archivo_txt):
  '''funcion para leer un archivo en formato txt'''
  with open(archivo_txt, "r", encoding="utf8") as f:
    texto = f.read()
    texto = texto.replace("\n", "").replace("\r", "") 
  return texto

In [4]:
def probabilidad_unigrama(token):
    if token not in unigramas:
        return 0
    return unigramas[token] / total_tokens
    
def get_prob_bigram(secuencia):
    if secuencia not in bigramas:
        return 0
    return bigramas[secuencia] / unigramas[secuencia[0]]
    
def get_prob_trigram(secuencia):
    if secuencia not in trigramas:
        return 0
    return trigramas[secuencia] / bigramas[secuencia[:2]]

def selecciona_siguiente_token(secuencia):
    lista_tokens_candidatos = []
    
    # Itera por todos los tokens del vocabulario
    # calculando la probabilidad de seleccionar la siguiente palabra
    for token in vocab:
        p1 = probabilidad_unigrama((token))
        p2 = get_prob_bigram((secuencia[-1], token))
        p3 = get_prob_trigram((secuencia[-2], secuencia[-1], token)) if len(secuencia) >= 3 else 0
        
        # Usamos una interpolación lineal
        p =  0.01*p1 + 0.35*p2 + 0.6*p3
        
        lista_tokens_candidatos.append((token, p))
    
    # orenamos la lista de mayor a menor probabilidad
    lista_tokens_candidatos.sort(key=lambda x: x[1], reverse=True)
    return lista_tokens_candidatos[0][0]
   


# Leer y limpiar texto

obtener archivo [txt](https://github.com/sharanyavb/harry-potter/blob/master/Books_Text/HP1.txt) 

In [39]:
libro = leer_libro('/content/HP1.txt')

libro[:1500]

'THE BOY WHO LIVED Mr. and Mrs. Dursley, of number four, Privet Drive, were proud to say that they were perfectly normal, thank you very much. They were the last people you’d expect to be involved in anything strange or mysterious, because they just didn’t hold with such nonsense. Mr. Dursley was the director of a firm called Grunnings, which made drills. He was a big, beefy man with hardly any neck, although he did have a very large mustache. Mrs. Dursley was thin and blonde and had nearly twice the usual amount of neck, which came in very useful as she spent so much of her time craning over garden fences, spying on the neighbors. The Dursley s had a small son called Dudley and in their opinion there was no finer boy anywhere. The Dursleys had everything they wanted, but they also had a secret, and their greatest fear was that somebody would discover it. They didn’t think they could bear it if anyone found out about the Potters. Mrs. Potter was Mrs. Dursley’s sister, but they hadn’t P

In [41]:
# lista con cadenas correspondientes al pie de página.
borrar = ['Page | {} Harry Potter and the Philosophers Stone - J.K. Rowling'.format(i)\
         for i in range(2,347)]

for tx in borrar:
    libro = libro.replace(tx, '')

libro = libro.strip()

libro[:1500]

'THE BOY WHO LIVED Mr. and Mrs. Dursley, of number four, Privet Drive, were proud to say that they were perfectly normal, thank you very much. They were the last people you’d expect to be involved in anything strange or mysterious, because they just didn’t hold with such nonsense. Mr. Dursley was the director of a firm called Grunnings, which made drills. He was a big, beefy man with hardly any neck, although he did have a very large mustache. Mrs. Dursley was thin and blonde and had nearly twice the usual amount of neck, which came in very useful as she spent so much of her time craning over garden fences, spying on the neighbors. The Dursley s had a small son called Dudley and in their opinion there was no finer boy anywhere. The Dursleys had everything they wanted, but they also had a secret, and their greatest fear was that somebody would discover it. They didn’t think they could bear it if anyone found out about the Potters. Mrs. Potter was Mrs. Dursley’s sister, but they hadn’t  

# Tokenización

In [42]:
tokens = libro.split()
print('Total tokens en el libro', len(tokens))
tokens[0:20]

Total tokens en el libro 79155


['THE',
 'BOY',
 'WHO',
 'LIVED',
 'Mr.',
 'and',
 'Mrs.',
 'Dursley,',
 'of',
 'number',
 'four,',
 'Privet',
 'Drive,',
 'were',
 'proud',
 'to',
 'say',
 'that',
 'they',
 'were']

# N Gramas

In [43]:
unigramas = Counter(tokens)
total_tokens =len(unigramas)  # tokens unicos
vocab = list(unigramas.keys())

In [44]:
# OBTENER BIGRAMAS Y TRIGRAMAS CON SUS FRECUENCIAS

def obtener_ngramas(lista_tokens, N):
    ngramas = (tuple(lista_tokens[i:i+N]) for i in range(len(lista_tokens)-N+1))
    return ngramas
    
bigramas = Counter(obtener_ngramas(tokens,2))
trigramas = Counter(obtener_ngramas(tokens,3))

## Crear textos basadas en la probabilidad de ocurrencia

In [45]:
texto = []
longitud_texto = 40

# Selección aleatoria de la primera palabra de
rand_index = random.randint(0, total_tokens-1)
primer_token = vocab[rand_index]
texto.append(primer_token)

for _ in range(longitud_texto-1):
    palabra = selecciona_siguiente_token(texto)
    texto.append(palabra)
print (" ".join(texto))

evening, Dudley had been a bit of a large pink face, not much neck, small, watery blue eyes, and sat down on the other two had found out of the first time in his life. The Potters knew very well
